get the feature dataset first by preprocess.py

and replace the filename before load the dataset

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn import tree
from itertools import product
from sklearn.metrics import mean_squared_error
import graphviz 
import pickle

C:\Users\minhu\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [34]:
def get_multiple_days(model,scalers,features,labels,test_feature,seq):
    """
    scalers: (list of MinMaxScaler or sklearn.preprocessing.MinMaxScaler)
    features, labels: (list) already normed
    test_feture: (nparray)
    seq: (int) sequence size
    """
    pred=[]
    for i in range(28):
        # make a prediction for the next day
        pred.append(model.predict(test_feature))
        # add the prediction in features
        temp=scalers[0].inverse_transform(features[-1])
        temp[0:-1]=temp[1:]
        temp[-1]=0
        temp=scalers[0].transform(temp)
        temp[-1]=pred[-1]
        features.append(temp)
        #print(pred[-1],features[-1])
        labels.append(pred[-1])
    return scalers[1].inverse_transform(pred).reshape(1,-1)[0]

# Hyperparameter

In [12]:
depth=list(range(1,11,2))
depth.append(None)                                           
decision_params=[x for x in product(depth,list(range(2,15,2)),['best', 'random'])]

In [13]:
decision_params

[(1, 2, 'best'),
 (1, 2, 'random'),
 (1, 4, 'best'),
 (1, 4, 'random'),
 (1, 6, 'best'),
 (1, 6, 'random'),
 (1, 8, 'best'),
 (1, 8, 'random'),
 (1, 10, 'best'),
 (1, 10, 'random'),
 (1, 12, 'best'),
 (1, 12, 'random'),
 (1, 14, 'best'),
 (1, 14, 'random'),
 (3, 2, 'best'),
 (3, 2, 'random'),
 (3, 4, 'best'),
 (3, 4, 'random'),
 (3, 6, 'best'),
 (3, 6, 'random'),
 (3, 8, 'best'),
 (3, 8, 'random'),
 (3, 10, 'best'),
 (3, 10, 'random'),
 (3, 12, 'best'),
 (3, 12, 'random'),
 (3, 14, 'best'),
 (3, 14, 'random'),
 (5, 2, 'best'),
 (5, 2, 'random'),
 (5, 4, 'best'),
 (5, 4, 'random'),
 (5, 6, 'best'),
 (5, 6, 'random'),
 (5, 8, 'best'),
 (5, 8, 'random'),
 (5, 10, 'best'),
 (5, 10, 'random'),
 (5, 12, 'best'),
 (5, 12, 'random'),
 (5, 14, 'best'),
 (5, 14, 'random'),
 (7, 2, 'best'),
 (7, 2, 'random'),
 (7, 4, 'best'),
 (7, 4, 'random'),
 (7, 6, 'best'),
 (7, 6, 'random'),
 (7, 8, 'best'),
 (7, 8, 'random'),
 (7, 10, 'best'),
 (7, 10, 'random'),
 (7, 12, 'best'),
 (7, 12, 'random'),
 (7, 1

In [8]:
random_params=[x for x in product(list(range(2,8,2)),depth,list(range(2,10,2)))]
random_params

[(2, 1, 2),
 (2, 1, 4),
 (2, 1, 6),
 (2, 1, 8),
 (2, 3, 2),
 (2, 3, 4),
 (2, 3, 6),
 (2, 3, 8),
 (2, 5, 2),
 (2, 5, 4),
 (2, 5, 6),
 (2, 5, 8),
 (2, 7, 2),
 (2, 7, 4),
 (2, 7, 6),
 (2, 7, 8),
 (2, 9, 2),
 (2, 9, 4),
 (2, 9, 6),
 (2, 9, 8),
 (2, None, 2),
 (2, None, 4),
 (2, None, 6),
 (2, None, 8),
 (4, 1, 2),
 (4, 1, 4),
 (4, 1, 6),
 (4, 1, 8),
 (4, 3, 2),
 (4, 3, 4),
 (4, 3, 6),
 (4, 3, 8),
 (4, 5, 2),
 (4, 5, 4),
 (4, 5, 6),
 (4, 5, 8),
 (4, 7, 2),
 (4, 7, 4),
 (4, 7, 6),
 (4, 7, 8),
 (4, 9, 2),
 (4, 9, 4),
 (4, 9, 6),
 (4, 9, 8),
 (4, None, 2),
 (4, None, 4),
 (4, None, 6),
 (4, None, 8),
 (6, 1, 2),
 (6, 1, 4),
 (6, 1, 6),
 (6, 1, 8),
 (6, 3, 2),
 (6, 3, 4),
 (6, 3, 6),
 (6, 3, 8),
 (6, 5, 2),
 (6, 5, 4),
 (6, 5, 6),
 (6, 5, 8),
 (6, 7, 2),
 (6, 7, 4),
 (6, 7, 6),
 (6, 7, 8),
 (6, 9, 2),
 (6, 9, 4),
 (6, 9, 6),
 (6, 9, 8),
 (6, None, 2),
 (6, None, 4),
 (6, None, 6),
 (6, None, 8)]

# train

## time features 

### time feature - load data

In [63]:
time_feature=pd.read_csv("df_time.csv")
time_feature.head()

,0,1,2,3,4,5,6,7,8,9,...,30492,30493,30494,30495,30496,30497,30498,30499,30500,30501
0,2011,1,29.0,1.0,1,1.0,1.0,1.0,1.0,0,...,0,14,1,0,4,0,0,0,0,0
1,2011,1,30.0,2.0,2,1.0,1.0,1.0,1.0,0,...,0,11,1,0,4,0,0,6,0,0
2,2011,1,31.0,3.0,3,1.0,1.0,1.0,1.0,0,...,0,5,1,0,2,2,0,0,0,0
3,2011,2,1.0,4.0,4,1.0,1.0,1.0,1.0,1,...,0,6,1,0,5,2,0,2,0,0
4,2011,2,2.0,5.0,5,1.0,1.0,1.0,1.0,1,...,0,5,1,0,2,0,0,2,0,0


In [25]:
X_train=time_feature.iloc[:-28,:12]
y_train=time_feature.iloc[:-28,12:]
X_test=time_feature.iloc[-28:,:12]
y_test=time_feature.iloc[-28:,12:]

In [28]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1913 entries, 0 to 1912
Columns: 30490 entries, 12 to 30501
dtypes: int64(30490)
memory usage: 445.0 MB


In [29]:
sc_x = MinMaxScaler(feature_range=(-1, 1))
X_train.iloc[:,:] = sc_x.fit_transform(X_train.iloc[:,:])
sc_y = MinMaxScaler(feature_range=(-1, 1))
y_train.iloc[:,:] = sc_y.fit_transform(y_train.iloc[:,:])

In [5]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-1.0,-1.000000,0.866667,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,-1.0,-1.000000,0.933333,-0.998954,-0.666667,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,-1.0,-1.000000,1.000000,-0.997908,-0.333333,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,-1.0,-0.818182,-1.000000,-0.996862,0.000000,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0
4,-1.0,-0.818182,-0.933333,-0.995816,0.333333,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0


In [30]:
#X_train.to_csv("df_time_normXtrain.csv",index=False)
#y_train.to_csv("df_time_normytrain.csv",index=False)

In [8]:
joblib.dump(sc_x, "time_normX.save")
joblib.dump(sc_y, "time_normy.save")

['time_normy.save']

In [55]:
#X_train=pd.read_csv("df_time_normXtrain.csv")
#y_train=pd.read_csv("df_time_normytrain.csv")

In [64]:
X_test=time_feature.iloc[-28:,:12]
y_test=time_feature.iloc[-28:,12:]

In [60]:
#sc_x = joblib.load("time_normX.save") 
#sc_y = joblib.load("time_normy.save") 

In [65]:
X_test.iloc[:,:]=sc_x.transform(X_test.iloc[:,:])
y_test.iloc[:,:]=sc_y.transform(y_test.iloc[:,:])

In [66]:
#X_test.to_csv("df_time_normXtest.csv",index=False)
#y_test.to_csv("df_time_normytest.csv",index=False)

In [56]:
#X_test=pd.read_csv("df_time_normXtest.csv")
#y_test=pd.read_csv("df_time_normytest.csv")

In [57]:
y_train.shape # (1913, 30490)

(1913, 30490)

In [67]:
y_test.shape # (28, 30490)

(28, 30490)

In [68]:
X_test.shape # (28, 12)

(28, 12)

### Decision Tree

In [69]:
score=[] # [r2,mse]
for row in decision_params:
    #print(row)
    decision=DecisionTreeRegressor(max_depth=row[0],min_samples_split=row[1],splitter=row[2],random_state=0)
    decision.fit(X_train,y_train)
    pred=decision.predict(X_test)
    score.append([decision.score(X_test,y_test),mean_squared_error(y_test,pred)])

In [70]:
score

[[-0.2133538981501621, 0.08687849123923105],
 [-0.19474330525573158, 0.08557217251291376],
 [-0.2133538981501621, 0.08687849123923105],
 [-0.19474330525573158, 0.08557217251291376],
 [-0.2133538981501621, 0.08687849123923105],
 [-0.19474330525573158, 0.08557217251291376],
 [-0.2133538981501621, 0.08687849123923105],
 [-0.19474330525573158, 0.08557217251291376],
 [-0.2133538981501621, 0.08687849123923105],
 [-0.19474330525573158, 0.08557217251291376],
 [-0.2133538981501621, 0.08687849123923105],
 [-0.19474330525573158, 0.08557217251291376],
 [-0.2133538981501621, 0.08687849123923105],
 [-0.19474330525573158, 0.08557217251291376],
 [-0.15224209072096737, 0.08259062058749066],
 [-0.1373510837905793, 0.08140310176119678],
 [-0.15224209072096737, 0.08259062058749066],
 [-0.1373510837905793, 0.08140310176119678],
 [-0.15224209072096737, 0.08259062058749066],
 [-0.1373510837905793, 0.08140310176119678],
 [-0.15224209072096737, 0.08259062058749066],
 [-0.1373510837905793, 0.08140310176119678],

In [71]:
max_ind=score.index(max(score))
max_ind

54

In [72]:
%%time
decision=DecisionTreeRegressor(max_depth=decision_params[max_ind][0],min_samples_split=decision_params[max_ind][1],splitter=decision_params[max_ind][2],random_state=0)
decision.fit(X_train,y_train)

Wall time: 8.05 s


DecisionTreeRegressor(criterion='mse', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=14, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [73]:
pred=decision.predict(X_test)
print("r2\t",decision.score(X_test,y_test),"\nmse\t",mean_squared_error(y_test,pred))

r2	 -0.12011218446004247 
mse	 0.07998052108338796


### Random Forest

In [74]:
score=[] # [r2,mse]
for row in random_params:
    #print(row)
    rf=RandomForestRegressor(n_estimators=row[0],max_depth=row[1],min_samples_split=row[2],random_state=0)
    rf.fit(X_train,y_train)
    pred=rf.predict(X_test)
    score.append([rf.score(X_test,y_test),mean_squared_error(y_test,pred)])

In [75]:
max_ind=score.index(max(score))
max_ind

56

In [76]:
%%time
rf=RandomForestRegressor(n_estimators=random_params[max_ind][0],max_depth=random_params[max_ind][1],min_samples_split=random_params[max_ind][2],random_state=0)
rf.fit(X_train,y_train)

Wall time: 21.6 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=6, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [77]:
pred=rf.predict(X_test)
print("r2\t",rf.score(X_test,y_test),"\nmse\t",mean_squared_error(y_test,pred))

r2	 -0.11158950156942116 
mse	 0.07945805772822152


## statistic

### statistic - load data

In [78]:
stat_feature=pd.read_csv("statistic_HOBBIES_1_004_CA_1.csv")
stat_feature.head(10)

,HOBBIES_1_004_CA_1,min_value,max_value,mean_value
0,0,NaN,NaN,NaN
1,0,NaN,NaN,NaN
2,0,NaN,NaN,NaN
3,0,NaN,NaN,NaN
4,0,NaN,NaN,NaN
5,0,NaN,NaN,NaN
6,0,NaN,NaN,NaN
7,0,0.0,0.0,0.0
8,0,0.0,0.0,0.0
9,0,0.0,0.0,0.0


In [79]:
sc = MinMaxScaler(feature_range=(-1, 1))
stat_feature.iloc[7:-28,:]=sc.fit_transform(stat_feature.iloc[7:-28,:])
stat_feature.iloc[-28:,:]=sc.transform(stat_feature.iloc[-28:,:])

In [80]:
X_train=stat_feature.iloc[7:-28,:1]
y_train=stat_feature.iloc[7:-28,1:]
X_test=stat_feature.iloc[-28:,:1]
y_test=stat_feature.iloc[-28:,1:]

### Decision Tree

In [83]:
score=[] # [r2,mse]
for row in decision_params:
    #print(row)
    decision=DecisionTreeRegressor(max_depth=row[0],min_samples_split=row[1],splitter=row[2],random_state=0)
    decision.fit(X_train,y_train)
    pred=decision.predict(X_test)
    score.append([decision.score(X_test,y_test),mean_squared_error(y_test,pred)])

In [84]:
score

[[-0.23032115480942558, 0.07089458294550006],
 [-0.23032115480942517, 0.07089458294550004],
 [-0.23032115480942558, 0.07089458294550006],
 [-0.23032115480942517, 0.07089458294550004],
 [-0.23032115480942558, 0.07089458294550006],
 [-0.23032115480942517, 0.07089458294550004],
 [-0.23032115480942558, 0.07089458294550006],
 [-0.23032115480942517, 0.07089458294550004],
 [-0.23032115480942558, 0.07089458294550006],
 [-0.23032115480942517, 0.07089458294550004],
 [-0.23032115480942558, 0.07089458294550006],
 [-0.23032115480942517, 0.07089458294550004],
 [-0.23032115480942558, 0.07089458294550006],
 [-0.23032115480942517, 0.07089458294550004],
 [-0.25441728874539077, 0.0722830703004572],
 [-0.23992458190661048, 0.07144795956284994],
 [-0.25441728874539077, 0.0722830703004572],
 [-0.23992458190661048, 0.07144795956284994],
 [-0.25441728874539077, 0.0722830703004572],
 [-0.23992458190661048, 0.07144795956284994],
 [-0.25441728874539077, 0.0722830703004572],
 [-0.23992458190661048, 0.071447959562

In [89]:
max_ind=score.index(max(score))
score[max_ind]

[-0.23015536227853106, 0.07088502951118959]

In [87]:
%%time
decision=DecisionTreeRegressor(max_depth=decision_params[max_ind][0],min_samples_split=decision_params[max_ind][1],splitter=decision_params[max_ind][2],random_state=0)
decision.fit(X_train,y_train)

Wall time: 3.99 ms


DecisionTreeRegressor(criterion='mse', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='random')

In [88]:
pred=decision.predict(X_test)
print("r2\t",decision.score(X_test,y_test),"\nmse\t",mean_squared_error(y_test,pred))

r2	 -0.23015536227853106 
mse	 0.07088502951118959


### Random Forest

In [90]:
score=[] # [r2,mse]
for row in random_params:
    #print(row)
    rf=RandomForestRegressor(n_estimators=row[0],max_depth=row[1],min_samples_split=row[2],random_state=0)
    rf.fit(X_train,y_train)
    pred=rf.predict(X_test)
    score.append([rf.score(X_test,y_test),mean_squared_error(y_test,pred)])

In [91]:
max_ind=score.index(max(score))
score[max_ind]

[-0.23681798328635756, 0.0712689485682808]

In [93]:
%%time
rf=RandomForestRegressor(n_estimators=random_params[max_ind][0],max_depth=random_params[max_ind][1],min_samples_split=random_params[max_ind][2],random_state=0)
rf.fit(X_train,y_train)

Wall time: 39.4 ms


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=1,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=6, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [94]:
pred=rf.predict(X_test)
print("r2\t",rf.score(X_test,y_test),"\nmse\t",mean_squared_error(y_test,pred))

r2	 -0.23681798328635756 
mse	 0.0712689485682808


## sequence

### sequence - load data

In [2]:
with open("seq_HOBBIES_1_004_CA_1_X.pickle", 'rb') as fp:
    X = pickle.load(fp)
    fp.close()
with open("seq_HOBBIES_1_004_CA_1_y.pickle", 'rb') as fp:
    y = pickle.load(fp)
    fp.close()

In [3]:
for i in range(len(X)):
    X[i]=list(X[i].astype(float))
    y[i]=float(y[i])

In [4]:
sc_X = MinMaxScaler(feature_range=(-1, 1))
X[:-28]=sc_X.fit_transform(X[:-28])
X[-28:]=sc_X.transform(X[-28:])

sc_y = MinMaxScaler(feature_range=(-1, 1))
y=list(np.array(y).reshape(-1,1))
y[:-28]=sc_y.fit_transform(y[:-28])
y[-28:]=sc_y.transform(y[-28:])
#y=list(np.array(y).reshape(1,-1))

In [5]:
for i in range(len(X)):
    X[i]=list(X[i])
    y[i]=list(y[i])

In [6]:
X_train=X[:-28]
y_train=y[:-28]
X_test=X[-28:]
y_test=y[-28:]

In [9]:
len(X_train[0])

28

### Decision Tree

In [44]:
score=[] # [r2,mse]
for row in decision_params:
    #print(row)
    decision=DecisionTreeRegressor(max_depth=row[0],min_samples_split=row[1],splitter=row[2],random_state=0)
    decision.fit(X_train,y_train)
    pred=decision.predict(X_test)
    score.append([decision.score(X_test,y_test),mean_squared_error(y_test,pred)])

In [45]:
score

[[0.22695822342950334, 0.042824060321127524],
 [-0.1258251887147479, 0.0623671414065789],
 [0.22695822342950334, 0.042824060321127524],
 [-0.1258251887147479, 0.0623671414065789],
 [0.22695822342950334, 0.042824060321127524],
 [-0.1258251887147479, 0.0623671414065789],
 [0.22695822342950334, 0.042824060321127524],
 [-0.1258251887147479, 0.0623671414065789],
 [0.22695822342950334, 0.042824060321127524],
 [-0.1258251887147479, 0.0623671414065789],
 [0.22695822342950334, 0.042824060321127524],
 [-0.1258251887147479, 0.0623671414065789],
 [0.22695822342950334, 0.042824060321127524],
 [-0.1258251887147479, 0.0623671414065789],
 [0.26370952731205655, 0.040788154756840056],
 [0.04512802517202663, 0.05289687606586711],
 [0.26370952731205655, 0.040788154756840056],
 [0.04512802517202663, 0.05289687606586711],
 [0.26370952731205655, 0.040788154756840056],
 [0.04512802517202663, 0.05289687606586711],
 [0.26370952731205655, 0.040788154756840056],
 [0.04512802517202663, 0.05289687606586711],
 [0.26

In [46]:
max_ind=score.index(max(score))
max_ind

14

In [47]:
score[max_ind]

[0.26370952731205655, 0.040788154756840056]

In [48]:
%%time
decision=DecisionTreeRegressor(max_depth=decision_params[max_ind][0],min_samples_split=decision_params[max_ind][1],splitter=decision_params[max_ind][2],random_state=0)
decision.fit(X_train,y_train)

Wall time: 44.9 ms


DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [49]:
pred=decision.predict(X_test)
print("r2\t",decision.score(X_test,y_test),"\nmse\t",mean_squared_error(y_test,pred))

r2	 0.26370952731205655 
mse	 0.040788154756840056


In [15]:
model=DecisionTreeRegressor(max_depth=decision_params[14][0],min_samples_split=decision_params[14][1],splitter=decision_params[14][2],random_state=0)
model.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=3, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='best')

In [35]:
get_multiple_days(model,[sc_X,sc_y],X_train,y_train,X_test[:1],28)

array([1.20828667, 1.20828667, 1.20828667, 1.20828667, 1.20828667,
       1.20828667, 1.20828667, 1.20828667, 1.20828667, 1.20828667,
       1.20828667, 1.20828667, 1.20828667, 1.20828667, 1.20828667,
       1.20828667, 1.20828667, 1.20828667, 1.20828667, 1.20828667,
       1.20828667, 1.20828667, 1.20828667, 1.20828667, 1.20828667,
       1.20828667, 1.20828667, 1.20828667])

### Random Forest

In [50]:
score=[] # [r2,mse]
for row in random_params:
    #print(row)
    rf=RandomForestRegressor(n_estimators=row[0],max_depth=row[1],min_samples_split=row[2],random_state=0)
    rf.fit(X_train,y_train)
    pred=rf.predict(X_test)
    score.append([rf.score(X_test,y_test),mean_squared_error(y_test,pred)])

In [51]:
max_ind=score.index(max(score))
max_ind

63

In [52]:
score[max_ind]

[0.1810630405639615, 0.04536650775288532]

In [53]:
%%time
rf=RandomForestRegressor(n_estimators=random_params[max_ind][0],max_depth=random_params[max_ind][1],min_samples_split=random_params[max_ind][2],random_state=0)
rf.fit(X_train,y_train)

Wall time: 136 ms


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=8,
           min_weight_fraction_leaf=0.0, n_estimators=6, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [54]:
pred=rf.predict(X_test)
print("r2\t",rf.score(X_test,y_test),"\nmse\t",mean_squared_error(y_test,pred))

r2	 0.1810630405639615 
mse	 0.04536650775288532


## stat multiple day

### stat multiple day - load data

In [96]:
X=pd.read_csv("stat_7_HOBBIES_1_004_CA_1.csv")
with open("stat_sales_HOBBIES_1_004_CA_1.pickle", 'rb') as fp:
    y = pickle.load(fp)
    fp.close()

In [99]:
i=0
for i in range(len(y)):
    if len(y[i])!=28:
        break


In [100]:
X=X[:i]
y=y[:i]

In [102]:
sc_X = MinMaxScaler(feature_range=(-1, 1))
X[:-28]=sc_X.fit_transform(X[:-28])
X[-28:]=sc_X.transform(X[-28:])

In [103]:
sc_y=MinMaxScaler(feature_range=(-1, 1))
y[:-28]=sc_y.fit_transform(y[:-28])
y[-28:]=sc_y.transform(y[-28:])

In [104]:
X_train=X[:-28]
y_train=y[:-28]
X_test=X[-28:]
y_test=y[-28:]

### Decision Tree

In [105]:
score=[] # [r2,mse]
for row in decision_params:
    #print(row)
    decision=DecisionTreeRegressor(max_depth=row[0],min_samples_split=row[1],splitter=row[2],random_state=0)
    decision.fit(X_train,y_train)
    pred=decision.predict(X_test)
    score.append([decision.score(X_test,y_test),mean_squared_error(y_test,pred)])

In [106]:
score

[[-0.01782176392258054, 0.06508767407241554],
 [-0.009365173517382296, 0.06454689196343785],
 [-0.01782176392258054, 0.06508767407241554],
 [-0.009365173517382296, 0.06454689196343785],
 [-0.01782176392258054, 0.06508767407241554],
 [-0.009365173517382296, 0.06454689196343785],
 [-0.01782176392258054, 0.06508767407241554],
 [-0.009365173517382296, 0.06454689196343785],
 [-0.01782176392258054, 0.06508767407241554],
 [-0.009365173517382296, 0.06454689196343785],
 [-0.01782176392258054, 0.06508767407241554],
 [-0.009365173517382296, 0.06454689196343785],
 [-0.01782176392258054, 0.06508767407241554],
 [-0.009365173517382296, 0.06454689196343785],
 [-0.025130246924820573, 0.0655550370002629],
 [-0.03231177869523185, 0.06601428164975023],
 [-0.025130246924820573, 0.0655550370002629],
 [-0.03231177869523185, 0.06601428164975023],
 [-0.025130246924820573, 0.0655550370002629],
 [-0.03231177869523185, 0.06601428164975023],
 [-0.025130246924820573, 0.0655550370002629],
 [-0.03231177869523185, 0.0

In [107]:
max_ind=score.index(max(score))
max_ind

1

In [108]:
score[max_ind]

[-0.009365173517382296, 0.06454689196343785]

In [109]:
%%time
decision=DecisionTreeRegressor(max_depth=decision_params[max_ind][0],min_samples_split=decision_params[max_ind][1],splitter=decision_params[max_ind][2],random_state=0)
decision.fit(X_train,y_train)

Wall time: 11 ms


DecisionTreeRegressor(criterion='mse', max_depth=1, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='random')

In [110]:
pred=decision.predict(X_test)
print("r2\t",decision.score(X_test,y_test),"\nmse\t",mean_squared_error(y_test,pred))

r2	 -0.009365173517382296 
mse	 0.06454689196343785


### Random Forest

In [111]:
score=[] # [r2,mse]
for row in random_params:
    #print(row)
    rf=RandomForestRegressor(n_estimators=row[0],max_depth=row[1],min_samples_split=row[2],random_state=0)
    rf.fit(X_train,y_train)
    pred=rf.predict(X_test)
    score.append([rf.score(X_test,y_test),mean_squared_error(y_test,pred)])

In [112]:
max_ind=score.index(max(score))
max_ind

48

In [113]:
score[max_ind]

[-0.01741466750339024, 0.06506164106743619]

In [114]:
%%time
rf=RandomForestRegressor(n_estimators=random_params[max_ind][0],max_depth=random_params[max_ind][1],min_samples_split=random_params[max_ind][2],random_state=0)
rf.fit(X_train,y_train)

Wall time: 38.6 ms


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=1,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=6, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [115]:
pred=rf.predict(X_test)
print("r2\t",rf.score(X_test,y_test),"\nmse\t",mean_squared_error(y_test,pred))

r2	 -0.01741466750339024 
mse	 0.06506164106743619


## seq multiple day

### seq multiple day - load data

In [120]:
with open("seq_m_HOBBIES_1_004_CA_1_X.pickle", 'rb') as fp:
    X = pickle.load(fp)
    fp.close()
with open("seq_m_HOBBIES_1_004_CA_1_y.pickle", 'rb') as fp:
    y = pickle.load(fp)
    fp.close()

In [121]:
sha=(len(X),len(X[0]),len(X[0][0]))
sc_X = MinMaxScaler(feature_range=(-1, 1))
X=np.array(X).reshape(sha[0],sha[1]*sha[2])
X[:-28]=sc_X.fit_transform(X[:-28])
X[-28:]=sc_X.transform(X[-28:])

In [122]:
sha=(len(y),len(y[0]),len(y[0][0]))
sc_y = MinMaxScaler(feature_range=(-1, 1))
y=np.array(y).reshape(sha[0],sha[1]*sha[2])
y[:-28]=sc_y.fit_transform(y[:-28])
y[-28:]=sc_y.transform(y[-28:])
#y=list(y.reshape(sha[0],sha[1],sha[2]))

In [123]:
for i in range(len(X)):
    X[i]=X[i].astype(float)
    y[i]=y[i].astype(float)

In [124]:
X=X.astype(float)
y=y.astype(float)

In [125]:
X_train=X[:-28]
y_train=y[:-28]
X_test=X[-28:]
y_test=y[-28:]

### Decision Tree

In [126]:
score=[] # [r2,mse]
for row in decision_params:
    #print(row)
    decision=DecisionTreeRegressor(max_depth=row[0],min_samples_split=row[1],splitter=row[2],random_state=0)
    decision.fit(X_train,y_train)
    pred=decision.predict(X_test)
    score.append([decision.score(X_test,y_test),mean_squared_error(y_test,pred)])

In [127]:
score

[[-0.015013895372289488, 0.06511197413859282],
 [0.005784488322630273, 0.0637777817423681],
 [-0.015013895372289488, 0.06511197413859282],
 [0.005784488322630273, 0.0637777817423681],
 [-0.015013895372289488, 0.06511197413859282],
 [0.005784488322630273, 0.0637777817423681],
 [-0.015013895372289488, 0.06511197413859282],
 [0.005784488322630273, 0.0637777817423681],
 [-0.015013895372289488, 0.06511197413859282],
 [0.005784488322630273, 0.0637777817423681],
 [-0.015013895372289488, 0.06511197413859282],
 [0.005784488322630273, 0.0637777817423681],
 [-0.015013895372289488, 0.06511197413859282],
 [0.005784488322630273, 0.0637777817423681],
 [0.040683733067268545, 0.06153903532556574],
 [0.009235731294093304, 0.06355638847462226],
 [0.040683733067268545, 0.06153903532556574],
 [0.009235731294093304, 0.06355638847462226],
 [0.040683733067268545, 0.06153903532556574],
 [0.009235731294093304, 0.06355638847462226],
 [0.040683733067268545, 0.06153903532556574],
 [0.009235731294093304, 0.06355638

In [128]:
max_ind=score.index(max(score))
max_ind

55

In [129]:
score[max_ind]

[0.05553826300344503, 0.06058613431262887]

In [130]:
%%time
decision=DecisionTreeRegressor(max_depth=decision_params[max_ind][0],min_samples_split=decision_params[max_ind][1],splitter=decision_params[max_ind][2],random_state=0)
decision.fit(X_train,y_train)

Wall time: 18.2 ms


DecisionTreeRegressor(criterion='mse', max_depth=7, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=14, min_weight_fraction_leaf=0.0,
           presort=False, random_state=0, splitter='random')

In [131]:
pred=decision.predict(X_test)
print("r2\t",decision.score(X_test,y_test),"\nmse\t",mean_squared_error(y_test,pred))

r2	 0.05553826300344503 
mse	 0.06058613431262887


### Random Forest

In [132]:
score=[] # [r2,mse]
for row in random_params:
    #print(row)
    rf=RandomForestRegressor(n_estimators=row[0],max_depth=row[1],min_samples_split=row[2],random_state=0)
    rf.fit(X_train,y_train)
    pred=rf.predict(X_test)
    score.append([rf.score(X_test,y_test),mean_squared_error(y_test,pred)])

In [133]:
max_ind=score.index(max(score))
max_ind

59

In [134]:
score[max_ind]

[0.08446534839589313, 0.05873049505038298]

In [135]:
%%time
rf=RandomForestRegressor(n_estimators=random_params[max_ind][0],max_depth=random_params[max_ind][1],min_samples_split=random_params[max_ind][2],random_state=0)
rf.fit(X_train,y_train)

Wall time: 36.4 ms


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=8,
           min_weight_fraction_leaf=0.0, n_estimators=6, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [137]:
pred=rf.predict(X_test)
print("r2\t",rf.score(X_test,y_test),"\nmse\t",mean_squared_error(y_test,pred))

r2	 0.08446534839589313 
mse	 0.05873049505038298
